In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib notebook

In [2]:
import sys,re,collections

from oauth2client import client
from googleapiclient import sample_tools

In [5]:
class PI:
    PI_list = collections.defaultdict(list)
    proj_list = collections.defaultdict(str)
    def __init__(self, **kwargs):
        self.name = kwargs.get('name', '')
        self.dept = kwargs.get('dept', [])
        self.proj = kwargs.get('proj', [])
        for p in self.proj:
            if not p in PI.PI_list[self.name]:
                PI.PI_list[self.name].append(p)
                
            if not p in PI.proj_list.keys():
                PI.proj_list[p] = self.name
    
    def __repr__(self):
        return 'PI: {} '.format(self.name) + ','.join(self.dept) + ' ' + ','.join(self.proj)
    
    def add_dept(s):
        self.dept.append(s)
        
    def add_proj(self, s):
        self.proj.append(s)
    
    @classmethod
    def get_PI_list(cls):
        return cls.PI_list
    
    @classmethod
    def get_proj_list(cls):
        return cls.proj_list


In [8]:
LuiSha = PI(name='Lui Sha',dept=['CS'],proj=['2016-08-100-02'])
RomitChoudhury = PI(name='Romit Roy Choudhury',dept=['ECE'],proj=['2017-06-100-01'])
SayanMitra = PI(name='Sayan Mitra',dept=['ECE'],proj=['2016-06-103-01'])
GirishKrishnan = PI(name='Girish Krishnan',dept=['ISE'],proj=['2017-03-106-01'])
SethHutchinson = PI(name='Seth Hutchinson', dept=['ECE'],proj=['2016-10-103-02', '2016-10-103-01'])
GraceGao = PI(name='Grace Gao', dept=['AE'], proj=['2016-11-103-01'])
MinhDo = PI(name='Minh Do', dept=['ECE'], proj=['2016-11-103-02'])
TimBretl = PI(name='Timothy Bretl', dept=['AE'], proj=['2017-02-103-02'])
MichaelSelig = PI(name='Michael Selig', dept=['AE'], proj=['2017-02-102-01'])
HwPark = PI(name='Hae-Won Park', dept=['ME'], proj=['2017-02-101-01'])
AimyWissa = PI(name='Aimy Wissa', dept=['ME'], proj=['2017-04-101-01'])
SayanMitra = PI(name='Sayan Mitra', dept=['ECE'], proj=['2017-03-103-01'])
PhilipAnsell = PI(name='Philip Ansell', dept=['AE'], proj=['2017-01-102-01'])
RomitRoyChoudhury = PI(name='Romit Roy Choudhury', dept=['ECE'], proj=['2016-08-100-01', '2017-02-100-01'])
GirishChowdhary = PI(name='Girish Chowdhary', dept=['ABE'], proj=['2016-11-105-01', '2016-10-105-01'])
AmyLaViers = PI(name='Amy LaViers', dept=['ME'], proj=['2016-12-101-01'])
NairaHovakimyan = PI(name='Naira Hovakimyan', dept=['ME'], proj=['2016-08-101-01'])


proj_number = PI.get_proj_list()
PIs = PI.get_PI_list()
    
print(PIs)

print('\n')
print(proj_number)

defaultdict(<class 'list'>, {'Lui Sha': ['2016-08-100-02'], 'Romit Roy Choudhury': ['2017-06-100-01', '2016-08-100-01', '2017-02-100-01'], 'Sayan Mitra': ['2016-06-103-01', '2017-03-103-01'], 'Girish Krishnan': ['2017-03-106-01'], 'Seth Hutchinson': ['2016-10-103-02', '2016-10-103-01'], 'Grace Gao': ['2016-11-103-01'], 'Minh Do': ['2016-11-103-02'], 'Timothy Bretl': ['2017-02-103-02'], 'Michael Selig': ['2017-02-102-01'], 'Hae-Won Park': ['2017-02-101-01'], 'Aimy Wissa': ['2017-04-101-01'], 'Philip Ansell': ['2017-01-102-01'], 'Girish Chowdhary': ['2016-11-105-01', '2016-10-105-01'], 'Amy LaViers': ['2016-12-101-01'], 'Naira Hovakimyan': ['2016-08-101-01']})


defaultdict(<class 'str'>, {'2016-08-100-02': 'Lui Sha', '2017-06-100-01': 'Romit Roy Choudhury', '2016-06-103-01': 'Sayan Mitra', '2017-03-106-01': 'Girish Krishnan', '2016-10-103-02': 'Seth Hutchinson', '2016-11-103-01': 'Grace Gao', '2016-11-103-02': 'Minh Do', '2017-02-103-02': 'Timothy Bretl', '2017-02-102-01': 'Michael Seli